##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Avro Dataset API

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/io/tutorials/avro"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/burgerkingeater/io/blob/vailidate_tutorial/docs/tutorials/avro.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/burgerkingeater/io/blob/vailidate_tutorial/docs/tutorials/avro.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
      <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/io/docs/tutorials/avro.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

The objective of Avro Dataset API is to load Avro formatted data natively into TensorFlow. 


### Install required Packages

In [ ]:
!pip install tensorflow-io

## Usage

Download a sample Avro file:

In [ ]:
!curl -OL https://github.com/burgerkingeater/io/raw/vailidate_tutorial/docs/tutorials/avro/mnist.avron!ls -l mnist.avro

Download the schema file corresponding for the sample Avro data:

In [ ]:
!curl -OL https://github.com/burgerkingeater/io/raw/vailidate_tutorial/docs/tutorials/avro/mnist.avsc
!ls -l mnist.avsc

In [ ]:
import tensorflow as tf
import tensorflow_io as tfio

features = {
    'features[*]': tfio.experimental.columnar.VarLenFeatureWithRank(dtype=tf.int32),
    'label': tf.io.FixedLenFeature(shape=[], dtype=tf.int32),
    'dataType': tf.io.FixedLenFeature(shape=[], dtype=tf.string)
}

schema = tf.io.gfile.GFile('mnist.avsc').read()

dataset = tfio.experimental.columnar.make_avro_record_dataset(file_pattern=['mnist.avro'],
                                                              reader_schema=schema,
                                                              features=features,
                                                              batch_size=2,
                                                              num_epochs=1)

print(next(dataset))


In the above example, The avro file `mnist.avro` has 3 fields for each record: `features`, which is an array of int, `label`, an int, and `dataType`, an enum. Here is a sample record from `mnist.avro`
<table class="avro-record-sample" align="center">
  <td> <a target="_blank"><img src="https://github.com/burgerkingeater/io/raw/vailidate_tutorial/docs/tutorials/avro/one_record_in_mnist.png">first record in mnist.avro</a>
  </td>
</table>
And the schema of `mnist.avro` which is represented by `mnist.avsc` is 
<table class="avro-schema" align="center">
  <td> <a target="_blank"><img src="https://github.com/burgerkingeater/io/raw/vailidate_tutorial/docs/tutorials/avro/avro_schema.png" width="300" height="300">the schema of mnist.avro</a>
  </td>
</table>


The above example converts `features`, `label`, `dataType` field to a VarLenFeature, FixedLenFeature, and FixedLenFeature respectively.
Please refer to <a target="_blank" href="https://www.tensorflow.org/io/api_docs/python/tfio/experimental/columnar/make_avro_record_dataset">API doc</a> for the detailed usage of `make_avro_record_dataset.`


The avro dataset can parse and coerce any avro data into TensorFlow tensors, including records in records, maps, arrays, branches, and enumerations. The parsing information is passed into the avro dataset implementation as a map where 
keys encode how to parse the data 
values encode on how to coerce the data into TensorFlow tensors – deciding the primitive type (e.g. bool, int, long, float, double, string) as well as the tensor type (e.g. sparse or dense). A listing of TensorFlow's parser types (see Table 1) and the coercion of primitive types (Table 2) is provided. 

Table 1:

TensorFlow Parser Types|TensorFlow Tensors|Explanation
----|----|------
tf.FixedLenFeature([], tf.int32)|dense tensor|Parse a fixed length feature; that is all rows have the same constant number of elements, e.g. just one element or an array that has always the same number of elements for each row 
tf.SparseFeature(index_key=['key_1st_index', 'key_2nd_index'], value_key='key_value', dtype=tf.int64, size=[20, 50]) |sparse tensor|Parse a sparse feature where each row has a variable length list of indices and values. The 'index_key' identifies the indices. The 'value_key' identifies the value. The 'dtype' is the data type. The 'size' is the expected maximum index value for each index entry
tf.VarLenFeature([],tf.int64) |sparse tensor|Parse a variable length feature; that means each data row can have a variable number of elements, e.g. the 1st row has 5 elements, the 2nd row has 7 elements

Table 2:

Avro Primitive Type|TensorFlow Primitive Type
----|----
boolean: a binary value|tf.bool
bytes: a sequence of 8-bit unsigned bytes|tf.string
double: double precision 64-bit IEEE floating point number|tf.float64
enum: enumeration type|tf.string using the symbol name
float: single precision 32-bit IEEE floating point number|tf.float32
int: 32-bit signed integer|tf.int32
long: 64-bit signed integer|tf.int64
null: no value|uses default value
string: unicode character sequence|tf.string


A comprehensive set of examples of Avro dataset API is provided within <a target="_blank" href="https://github.com/tensorflow/io/blob/vailidate_tutorial/tests/test_parse_avro_eager.py#L580r">the tests</a>
